In [1]:
print('''STUFF TO KEEP AT THE BACK OF MIND
Below are methods to find a web element based on the locators in Selenium python bindings:
find_element_by_id.
find_element_by_name.
find_element_by_xpath.
find_element_by_link_text.
find_element_by_partial_link_text.
find_element_by_tag_name.
find_element_by_class_name.
find_element_by_css_selector.
LINKEDIN - https://www.linkedin.com/sales/gmail/profile/viewByEmail/
UNLISTED PASTES - https://netbootcamp.org/pastesearch.html#gsc.tab=0 
TIKTOK - https://www.tiktok.com/node/share/user/@UserName
use X-Rate-Limit-Daily-Remaining header to see remaining searches on emailrep.io
If successfully find username use - https://github.com/woj-ciech/SocialPath.git
''')

STUFF TO KEEP AT THE BACK OF MIND
Below are methods to find a web element based on the locators in Selenium python bindings:
find_element_by_id.
find_element_by_name.
find_element_by_xpath.
find_element_by_link_text.
find_element_by_partial_link_text.
find_element_by_tag_name.
find_element_by_class_name.
find_element_by_css_selector.
LINKEDIN - https://www.linkedin.com/sales/gmail/profile/viewByEmail/
UNLISTED PASTES - https://netbootcamp.org/pastesearch.html#gsc.tab=0 
TIKTOK - https://www.tiktok.com/node/share/user/@UserName
use X-Rate-Limit-Daily-Remaining header to see remaining searches on emailrep.io
If successfully find username use - https://github.com/woj-ciech/SocialPath.git



In [3]:
#NEEDED LIBRARIES
import sys
from os import path
from requests import get,session,packages
from bs4 import BeautifulSoup as bs,Comment

#!pip3 install webdriver_manager
#Just webdriver shit
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys


import json 
import re
import csv
from time import sleep
from urllib.parse import unquote

#The next two lines are just to disable the stupid insecure request warning
from urllib3.exceptions import InsecureRequestWarning
packages.urllib3.disable_warnings(category = InsecureRequestWarning)

#options = webdriver.ChromeOptions()
#options.add_argument("headless")
#options.add_argument('window-size=1200x600');

#MAJOR WEBSITES
twitter_url = "https://twitter.com"
insta_url = "https://www.instagram.com"
fb_url = "https://www.facebook.com/public"
hibp= "https://haveibeenpwned.com/unifiedsearch"
email_rep="https://emailrep.io"

check = '^[a-z0-9]+[\._]?[a-z0-9]+@\w+\.\w{2,3}$'
ran_once=False
target_email=""
while not re.search(check, target_email):
    target_email=input("Please enter target email : ")

Please enter target email : kavita.shivashankar@gmail.com


In [3]:
#DOWNLOAD FUNCTION
def download(username,site_name,dp_url): 
    count = 0 
    filename = f"{username}'s_{site_name}_dp.jpg" 
    while True: 
        if not path.isfile(filename): 
            with open(filename, 'wb+') as handle: 
                response = get(dp_url, stream=True)
                if not response.ok: 
                    print(response)
                for block in response.iter_content(1024): 
                    if not block: 
                        break 
                    handle.write(block) 
        else: 
            count += 1 
            filename = f"{username}'s_{site_name}_dp_{count}.jpg" 
            continue 
        print(f"{username}'s {site_name} dp downloaded!")
        break

In [4]:
#HAVE I BEEN PWNED
#email=input("Enter your email-id : ").lower().replace("@","%40")
#target_email="mansirao2001@gmail.com"
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
print("Initialized Headless")
print()
driver.get(f"{hibp}/{target_email}")
soup = bs(driver.page_source)
data = json.loads(soup.find("body").text)
print(json.dumps(data,indent = 4).replace("\\\"","'"))

Initialized Headless

{
    "Breaches": [
        {
            "Name": "Dubsmash",
            "Title": "Dubsmash",
            "Domain": "dubsmash.com",
            "BreachDate": "2018-12-01",
            "AddedDate": "2019-02-25T08:35:58Z",
            "ModifiedDate": "2019-02-25T08:35:58Z",
            "PwnCount": 161749950,
            "Description": "In December 2018, the video messaging service <a href='https://www.theregister.co.uk/2019/02/11/620_million_hacked_accounts_dark_web/' target='_blank' rel='noopener'>Dubsmash suffered a data breach</a>. The incident exposed 162 million unique email addresses alongside usernames and PBKDF2 password hashes. In 2019, the data appeared listed for sale on a dark web marketplace (along with several other large breaches) and subsequently began circulating more broadly. The data was provided to HIBP by a source who requested it to be attributed to &quot;BenjaminBlue@exploit.im&quot;.",
            "LogoPath": "https://haveibeenpwned.com/Cont

In [5]:
#EMAIL REP
#email="mansirao2001@gmail.com"
headers={
    "User-Agent" : "python/emailrep.io",
    "Content-Type" : "application/json",
    "Key" : "9u880v5bumdu7m37c9zipnv3kwyq1bb3izmuxpwjjvrj3gnv"
}
email_rep_data = get(f"{email_rep}/{target_email}",headers=headers).content
email_rep_data = json.loads(email_rep_data)
print(json.dumps(email_rep_data, indent = 2))

{
  "email": "contact.shiva@gmail.com",
  "reputation": "high",
  "suspicious": false,
  "references": 10,
  "details": {
    "blacklisted": false,
    "malicious_activity": false,
    "malicious_activity_recent": false,
    "credentials_leaked": true,
    "credentials_leaked_recent": false,
    "data_breach": true,
    "first_seen": "05/05/2012",
    "last_seen": "02/20/2020",
    "domain_exists": true,
    "domain_reputation": "n/a",
    "new_domain": false,
    "days_since_domain_creation": 9060,
    "suspicious_tld": false,
    "spam": false,
    "free_provider": true,
    "disposable": false,
    "deliverable": true,
    "accept_all": false,
    "valid_mx": true,
    "spoofable": true,
    "spf_strict": true,
    "dmarc_enforced": false,
    "profiles": [
      "spotify",
      "pinterest",
      "twitter"
    ]
  }
}


In [6]:
#SHERLOCK
from sherlock import sherlock
sites_we_need = [
                 "Facebook",
                 "Instagram",
                 "Twitter",
                 "Linkedin",
                 "Medium",
    
                ]
try:
    sites_we_need.extend(list(i.title() for i in email_rep_data["details"]["profiles"]))
except Exception as e:
    pass
data = open("new_data.json")
sites = json.load(data)
username = target_email[:target_email.index('@')].replace(".","")
results = sherlock(username, sites, print_found_only=True, color=False)
exists = []
with open(username + ".csv", "w", newline='', encoding="utf-8") as csv_report:
    writer = csv.writer(csv_report)
    for site in results:
        if site in sites_we_need and results[site]['http_status']!=404:
            exists.append(site)
            towrite=[site,results[site]['url_user']]
            print(results[site]['url_user'])
            writer.writerow(towrite)

https://www.facebook.com/contactshiva
https://www.instagram.com/contactshiva
https://www.pinterest.com/contactshiva/
https://www.twitter.com/contactshiva


In [5]:
#Getting Name from email
#Logging into Google
#target_email="contact.shiva@icloud.com"
#options = Options()
#options.headless = True
print(target_email)
wait = WebDriverWait(driver, 20)
driver.get('https://stackoverflow.com/users/signup?ssrc=head&returnurl=%2fusers%2fstory%2fcurrent%27')
print("Headless initialized")
sleep(5)
driver.find_element_by_xpath('//*[@id="openid-buttons"]/button[1]').click()
driver.find_element_by_xpath('//input[@type="email"]').send_keys('mydbmsproject2001@gmail.com'+Keys.ENTER)
print("email done")
sleep(3)
driver.find_element_by_xpath('//input[@type="password"]').send_keys('g00dpassw0rd'+Keys.ENTER)
wait.until(EC.presence_of_element_located((By.ID, "content")))
print("logged in")
sleep(2)
#Adding target as contact to google contacts
driver.get("https://contacts.google.com/")
print("jumped to contacts")
sleep(2)
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@title="Add new contact"]'))).click()
#driver.find_element_by_xpath('//*[@title="Add new contact"]').click()
sleep(2)
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Create a contact"]'))).click()
#wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'whsOnd zHQkBf'))).send_keys(target_email)
wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div[4]/div/div[2]/span/div/div[2]/div[1]/div/div/div[6]/div/div/div[2]/div[1]/div[1]/div/div[1]/input'))).send_keys(target_email)
#driver.find_element_by_xpath('/html/body/div[7]/div[4]/div/div[2]/span/div/div[2]/div[1]/div/div/div[6]/div/div/div[2]/div[1]/div[1]/div/div[1]/input').send_keys(target_email)
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@jsname="x8hlje"]'))).click()
sleep(2)
try:
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ONOzI")))
except:
    driver.save_screenshot('test.png')
page = driver.page_source
target_contact_soup = bs(page,'html.parser')
try:
    target_id = target_contact_soup.find('div',{"class":"NVFbjd LAORIe"}).attrs['data-sourceid']
    print("Added contact and got google maps url")


    # Bounce to maps and get name
    maps_url = f"https://www.google.com/maps/contrib/{target_id}"
    print("jumped to maps")
    print("Google maps url : ",maps_url)
    driver.get(f"https://www.google.com/maps/contrib/{target_id}")
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
    name_page=driver.page_source
    name_soup=bs(name_page,'html.parser')
    name=name_soup.find("h1",{"role":"button","class":"section-profile-header-name section-profile-header-clickable-item"}).text
    print("Name : ",name)
except Exception as e:
    driver.save_screenshot('error.png')
    print("Sorry this email is not linked to a google account")

kavita.shivashankar@gmail.com
Headless initialized
email done
logged in
jumped to contacts
Added contact and got google maps url
jumped to maps
Google maps url :  https://www.google.com/maps/contrib/111747508985759040865
Name :  Kavita Sivashankar


In [8]:
#INSTA INFO
if "Instagram" in exists:
    #options = Options()
    #options.headless = True
    #driver = webself.driver.Firefox(options=options)
    wait = WebDriverWait(driver, 20)
    self.driver.get(results["Instagram"]["url_user"])
    user_page = self.driver.page_source
    insta_soup = bs(user_page,'html.parser')
    if not insta_soup.find(text="Sorry, this page isn't available."):
        scripts = insta_soup.find('script', type="text/javascript", text=re.compile('window._sharedData'))
        stringified_json = scripts.text.replace('window._sharedData = ', '')[:-1]
        insta_data = json.loads(stringified_json)['entry_data']['ProfilePage'][0]
        insta_username = insta_data['graphql']['user']['username']
        if not path.isfile(f"{insta_username}'s_insta_data.json"):
            data_file = f"{insta_username}'s_insta_data.json" 
        else:
            data_file = f"{insta_username}'s_insta_data_1.json"
        #json.dump(insta_data, open(data_file,"w+"), indent=4)
        print(json.dumps(insta_data, indent=4))
    else:
        print("Sorry this insta account does not exist")

{
    "logging_page_id": "profilePage_257130609",
    "show_suggested_profiles": false,
    "show_follow_dialog": false,
    "graphql": {
        "user": {
            "biography": "",
            "blocked_by_viewer": false,
            "restricted_by_viewer": null,
            "country_block": false,
            "external_url": null,
            "external_url_linkshimmed": null,
            "edge_followed_by": {
                "count": 246
            },
            "followed_by_viewer": false,
            "edge_follow": {
                "count": 214
            },
            "follows_viewer": false,
            "full_name": "Shiva Shankar",
            "has_ar_effects": false,
            "has_channel": false,
            "has_blocked_viewer": false,
            "highlight_reel_count": 0,
            "has_requested_viewer": false,
            "id": "257130609",
            "is_business_account": false,
            "is_joined_recently": false,
            "business_category_name": 

In [9]:
#DOWNLOAD INSTA DP
dp_url = insta_data['graphql']['user']['profile_pic_url_hd']
filename = f"{insta_username}'s_insta_dp.jpg"
download(insta_username,"insta",dp_url)

contactshiva's insta dp downloaded!


In [10]:
#TWITTER INFO
if "Twitter" in exists:
    print(results["Twitter"]["url_user"])
    driver.get(results["Twitter"]["url_user"])
    sleep(5)
    page = driver.page_source
    html = bs(page,'html.parser')
    try:
#         # TODO: Check what kind of exception raising if no location
#         location = html.find("span",{"class":"ProfileHeaderCard-locationText u-dir"}).text
#         birthday = html.find("span",{"class":"ProfileHeaderCard-birthdateText u-dir"}).text.strip()
#         if birthday:
#            birthday = birthday.replace('Born ', '')
#         else:
#             birthday = None
          #profile_photo = html.find("img",{"class":"ProfileAvatar-image"}).attrs['src']
#         page_title = html.find('title').text
#         name = page_title[:page_title.find('(')].strip()
#         biography = html.find("p",{"class":"ProfileHeaderCard-bio u-dir"}).text
#         #website = html.find("a",{"class":"u-textUserColor","rel":"me nofollow noopener"}).text.strip()
#         print(json.dumps(dict(
#         twitter_name = name,
#         twitter_username = username,
#         twitter_birthday = birthday,
#         biography = biography,
#         profile_photo = profile_photo ), indent=4))
#         #download(twit_username,"twitter",profile_photo)

          location = html.find("span",{"class":"css-901oao css-16my406 r-111h2gw r-4qtqp9 r-1qd0xha r-ad9z0x r-zso239 r-bcqeeo r-qvutc0"}).text  
        
    except:
        print("Account is private.")

https://www.twitter.com/contactshiva


In [15]:
#FACEBOOK INFO
if "Facebook" in exists:
    print(results["Facebook"]["url_user"])
    driver.get(results["Facebook"]["url_user"])
    sleep(3)
    page = driver.page_source
    html = bs(page,'html.parser')
    about_keys=list(map(lambda x:x.text,html.find_all("span",{"role":"heading"})))
    about_keys=about_keys[:about_keys.index("Contact Information")]
    about=list(map(lambda x:x.text,html.find_all("ul")))
    about=about[1:len(about_keys)+1]
    print(dict(zip(about_keys,about)))
    #print(list(map(lambda x:x.text,html.find_all("th",{"class":"label"}))))
    #print(list(map(lambda x:x.text,html.find_all("td",{"class":"data"}))))
    likes=dict(zip(list(map(lambda x:x.text,html.find_all("th",{"class":"label"}))),list(map(lambda x:x.text,html.find_all("td",{"class":"data"})))))
    print(likes)

https://www.facebook.com/contactshiva
{'Work': 'Reliance Jio Infocomm LimitedVice President · December 2014 to present · Navi Mumbai (New Mumbai), India'}
{'Music': 'TarunMotions Films & Photography', 'Books': 'Rules of the Game - www.rulesofthegame.in', 'Movies': 'RaghavaRao Productions', 'Television': 'Coke Studio', 'Athletes': 'MS Dhoni', 'Other': "IPL - Indian Premier League, east end apartments, HELLO! India, CIO, Flipkart, Curd Rice -ಬಕಳ ಬಾತ್ ,தயிர் சாதம் ,Thachi Mamu, TED, Ramakrishnan Aiyaswamy Photography, Lord Ganesha, The Ministry of Humor, Kalpesh Gandhi, Priti Henna Art, Vedanectar, TechNet India, Vishwajyot High School, ARIVU_arts, Oye teri aisi-taisi, Elitecisos, adStringO Software Pvt. Ltd., Computer Society of India Mumbai Chapter, The Growth Mindset, Dough&Batter, EPFO, Palghat Ramprasad, Wife and husband funny moments, IndiaCIOReview, CIO POWER LIST, Madras Street Food, Unofficial: Random Stories, CFO POWER LIST, Zeus School Of Music, Creation, Laugh Therapy, Jeevesh

In [ ]:
https://www.facebook.com/kaushikshivashankar
['Education', 'Current City and Home Town', 'Other Places Lived', 'About Kaushik', 'Favourite Quotes', 'Contact Information', 'Facebook']
['FriendsFriendsPhotosPhotosVideosVideos', 'PonvidyashramChennai, India', 'Mumbai, MaharashtraCurrent cityChennai, IndiaHome Town', 'Delhi, IndiaMoved here', 'I am Kaushik and I am more like a useful guy  ..', 'No favourite quotes to show', 'Shreya Ghoshal', 'Glu', 'Parkour', 'Kaushik Sivashankar', 'Kaushik BarotKaushik ModakKaushik DasKaushik SinghKaushik Goswami ChestaAkshat Kaushik', 'Facebookhttp://facebook.com/kaushik.shivashankar']